In [ ]:
# c24

from rdkit import Chem
from rdkit.Chem import AllChem

# 1. 输入 SMILES
smiles = "C1CC([C@H]3[C@@](C1)(C)[C@H]2CC[C@H](C)[C@H]([C@@]2(CC3)C)CCC(C)C)(C)C"

# 构建分子并加氢
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# 生成构象并优化
AllChem.EmbedMolecule(mol, randomSeed=42)
AllChem.MMFFOptimizeMolecule(mol)

conf = mol.GetConformer()

# === 写 GJF 文件 ===
gjf_lines = []

# Gaussian 路径前言
gjf_lines.append("%nproc=32")
gjf_lines.append("%mem=32GB")
gjf_lines.append("%Chk=C24-opt.chk")
gjf_lines.append("#p B3LYP/6-31G(d) Opt\n")
gjf_lines.append("Optimization of molecular geometry\n")
gjf_lines.append("0 1")

# 分子坐标
for atom in mol.GetAtoms():
    pos = conf.GetAtomPosition(atom.GetIdx())
    gjf_lines.append(f"{atom.GetSymbol():2s}  {pos.x:12.6f}  {pos.y:12.6f}  {pos.z:12.6f}")

# Link1 NBO 计算部分
gjf_lines.append("\n--Link1--")
gjf_lines.append("%OldChk=C24-opt.chk")
gjf_lines.append("%Chk=C24-nbo.chk")
gjf_lines.append("#n B3LYP/6-311+G(2d,p) POP=NBO Geom=Checkpoint\n")
gjf_lines.append("Calculations of NBO charges\n")
gjf_lines.append("0 1")

# 保存为 molecule.gjf
with open("C24.gjf", "w") as f:
    f.write("\n".join(gjf_lines))

print("[INFO] Gaussian 输入文件已生成：C24.gjf")
